xarray duck array support

outline:
- definition duck arrays
    - not: array_like (=castable with `np.asarray`)
    - needs to have:
        - `ndim`
        - `shape`
        - `dtype`
        - `__array_ufunc__`
        - `__array_function__` (NEP18)
    - future: `__array_module__` and `__array_namespace__`
    - examples:
        - pydata/sparse
        - pint
        - cupy
        - pytorch?
- duck dask arrays:
    - definition: duck array and dask collection
    - support status unknown (test suite is missing)
- extra features:
    - custom inline repr: `_repr_inline_`
- duck array testing framework:
    - using hypothesis
    - separated into parts (subclasses)  (inspired by pandas' extension array test framework)
    - can be used to test integration of new duck arrays, or even combinations, like:
      - `pint(dask(numpy))`
      - `pint(dask(sparse))`

In [ ]:
import xarray as xr
import sparse
import pint
import dask.array as da
import numpy as np

rng = np.random.default_rng()
ureg = pint.UnitRegistry()

# status of the duck array integration

- duck arrays
- integration status
- additional support
- testing framework

## duck arrays

*duck typing*:
> checking for interfaces instead of types (from the [python docs](https://docs.python.org/3/glossary.html#term-duck-typing))

*duck array*:
> same interface and semantics as `numpy.ndarray`, but with different behavior

*array_like*:
> anything converted to a `numpy.ndarray` when passed to `numpy.asarray`

*duck dask array*:
> *duck array* and *dask collection*

`xarray` requires these properties / methods: (see [Integrating with duck arrays](https://xarray.pydata.org/en/latest/internals/duck-arrays-integration.html))

- `ndim`, `shape`, `dtype`

- `__array__` (for plotting / `.values`, etc)

- `__array_ufunc__`, `__array_function__` (to support the `numpy` api)

- future: `__array_module__` and `__array_namespace__`

examples for duck arrays:
- `cupy`
- `dask`
- `pint`
- `sparse`

potential candidates:
- `pytorch`
- `pandas` extension arrays
- `awkward-array`

## integration status

see https://xarray.pydata.org/en/latest/user-guide/duckarrays.html

- most methods work
- exceptions:
    - indexing (might change with the index refactor)
    - external functions (`scipy`, `numbagg`, `bottleneck`)
    - functionality in `numpy` (`numpy.vectorize`)

### nested duck arrays

- it is possible to nest duck arrays (if they support it):

In [ ]:
arr_sparse = sparse.random((100, 100, 10), random_state=0)
arr_sparse

In [ ]:
arr_dask = da.from_array(arr_sparse, chunks=(10, 10, 10))
arr_dask

In [ ]:
arr_pint = ureg.Quantity(arr_dask, "m")
arr_pint

In [ ]:
arr = xr.DataArray(arr_pint, dims=("x", "y", "z"))
arr

In [ ]:
arr.coarsen({"x": 5, "y": 5, "z": 2}).mean()

- unsolved issues:
    - `repr`
    - construction of layers
    - interactions between duck arrays
    - editing specific layers (for example, convert `arr` to dense or chunk a `xarray(pint(sparse))` array)
    - testing

## `_repr_inline_`

- hook for custom reprs
- written to display units:

In [ ]:
import pint_xarray

with xr.set_options(display_expand_data=False):
    display(arr)

## duck array testing framework

motivation: each duck array behaves differently, support status unclear

`xarray.tests.duckarrays.base`
- using `hypothesis`
- inspired by `pandas`' ExtensionArray test framework
- separated into different parts of `xarray`'s API (using subclasses)